In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

In [2]:
# autoreload
%load_ext autoreload
%autoreload 2

In [7]:
my_run = BuildStockQuery('rescore','euss-oedi',
                                "resstock_amy2018_release_2022_1",
                                db_schema="resstock_oedi",
                                skip_reports=False)

INFO:buildstock_query.query_core:Loading resstock_amy2018_release_2022_1 ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:17 queries cache read from None.
INFO:buildstock_query.query_core:17 queries cache is updated.
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


         success  unapplicable  fail     Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                      
0         548916             0     0  548916        0.0        0         0   
1         531823         17093     0  548916       96.9        0         0   
10        548916             0     0  548916      100.0        0         0   
2         536797         12119     0  548916       97.8        0         0   
3         548260           656     0  548916       99.9        0         0   
4         548260           656     0  548916       99.9        0         0   
5         532156         16760     0  548916       96.9        0         0   
6         545331          3585     0  548916       99.3        0         0   
7         400579        148337     0  548916       73.0        0         0   
8         548913             3     0  548916      100.0        0         0   
9         548916             0     0  548916      100.0        0

INFO:buildstock_query.query_core:No new queries to save.


Annual and timeseries tables are verified to have the same number of buildings.
All buildings are verified to have the same number of (35040) timeseries rows.


In [8]:
baseline_agg = my_run.agg.aggregate_annual(enduses=['electricity.total.energy_consumption'],
                                       group_by=['geometry_building_type_recs'])
baseline_agg


,geometry_building_type_recs,sample_count,units_count,electricity.total.energy_consumption
0,Mobile Home,34491,8.351341e+06,9.934359e+10
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,8.623094e+10
2,Multi-Family with 5+ Units,100345,2.429664e+07,1.769924e+11
3,Single-Family Attached,32125,7.778459e+06,7.379245e+10
4,Single-Family Detached,338087,8.186135e+07,1.115071e+12


In [9]:
my_run.save_cache()

INFO:buildstock_query.query_core:No new queries to save.


In [10]:
all_enduses = my_run.get_cols(my_run.ts_table)
all_enduses

[Column('timestamp', TIMESTAMP(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.ceiling_fan.energy_consumption', FLOAT(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.ceiling_fan.energy_consumption_intensity', FLOAT(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.clothes_dryer.energy_consumption', FLOAT(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.clothes_dryer.energy_consumption_intensity', FLOAT(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.clothes_washer.energy_consumption', FLOAT(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.clothes_washer.energy_consumption_intensity', FLOAT(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.cooling_fans_pumps.energy_consumption', FLOAT(), table=<resstock_amy2018_release_2022_1_by_state>),
 Column('out.electricity.cooling_fans_pump

In [11]:
ts_agg = my_run.agg.aggregate_timeseries(enduses=['out.electricity.total.energy_consumption'],
                                         # restrict=[('build_existing_model.state', ['CO'])],
                                         timestamp_grouping_func='month',
                                         group_by=['geometry_building_type_recs', 'state'],
                                         get_query_only=False)
print(ts_agg)

INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


     geometry_building_type_recs state  timestamp  sample_count  \
0                    Mobile Home    AL 2018-01-01          1233   
1                    Mobile Home    AL 2018-02-01          1233   
2                    Mobile Home    AL 2018-03-01          1233   
3                    Mobile Home    AL 2018-04-01          1233   
4                    Mobile Home    AL 2018-05-01          1233   
...                          ...   ...        ...           ...   
2923      Single-Family Detached    WY 2018-08-01           725   
2924      Single-Family Detached    WY 2018-09-01           725   
2925      Single-Family Detached    WY 2018-10-01           725   
2926      Single-Family Detached    WY 2018-11-01           725   
2927      Single-Family Detached    WY 2018-12-01           725   

        units_count  rows_per_sample  electricity.total.energy_consumption  
0     298547.538693             2976                          5.748190e+08  
1     298547.538693             2688     

In [12]:
print(ts_agg)

     geometry_building_type_recs state  timestamp  sample_count  \
0                    Mobile Home    AL 2018-01-01          1233   
1                    Mobile Home    AL 2018-02-01          1233   
2                    Mobile Home    AL 2018-03-01          1233   
3                    Mobile Home    AL 2018-04-01          1233   
4                    Mobile Home    AL 2018-05-01          1233   
...                          ...   ...        ...           ...   
2923      Single-Family Detached    WY 2018-08-01           725   
2924      Single-Family Detached    WY 2018-09-01           725   
2925      Single-Family Detached    WY 2018-10-01           725   
2926      Single-Family Detached    WY 2018-11-01           725   
2927      Single-Family Detached    WY 2018-12-01           725   

        units_count  rows_per_sample  electricity.total.energy_consumption  
0     298547.538693             2976                          5.748190e+08  
1     298547.538693             2688     

In [13]:
result = my_run.savings.savings_shape(upgrade_id=2, enduses=['out.electricity.total.energy_consumption'],
                             annual_only=False,
                             get_query_only=False)
print(result)

                timestamp  sample_count   units_count  \
0     2018-01-01 00:15:00        548916  1.329096e+08   
1     2018-01-01 00:30:00        548916  1.329096e+08   
2     2018-01-01 00:45:00        548916  1.329096e+08   
3     2018-01-01 01:00:00        548916  1.329096e+08   
4     2018-01-01 01:15:00        548916  1.329096e+08   
...                   ...           ...           ...   
35035 2018-12-31 23:00:00        548916  1.329096e+08   
35036 2018-12-31 23:15:00        548916  1.329096e+08   
35037 2018-12-31 23:30:00        548916  1.329096e+08   
35038 2018-12-31 23:45:00        548916  1.329096e+08   
35039 2019-01-01 00:00:00        548916  1.329096e+08   

       electricity.total.energy_consumption__baseline  \
0                                        6.380506e+07   
1                                        6.390018e+07   
2                                        6.321905e+07   
3                                        6.284525e+07   
4                             

In [14]:
my_run.save_cache()

INFO:buildstock_query.query_core:No new queries to save.


In [15]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['out.electricity.total.energy_consumption'],
                             group_by=['geometry_building_type_recs'],
                             restrict=[('in.state', ['VT'])],
                             annual_only=False)

,timestamp,sample_count,units_count,electricity.total.energy_consumption__baseline,electricity.total.energy_consumption__savings,geometry_building_type_recs
0,2018-01-01 00:15:00,89,21549.660133,3332.933390,371.428974,Mobile Home
1,2018-01-01 00:30:00,89,21549.660133,3540.439668,363.438650,Mobile Home
2,2018-01-01 00:45:00,89,21549.660133,3993.708924,358.838161,Mobile Home
3,2018-01-01 01:00:00,89,21549.660133,3681.602049,355.690458,Mobile Home
4,2018-01-01 01:15:00,89,21549.660133,3638.744859,348.426527,Mobile Home
...,...,...,...,...,...,...
175195,2018-12-31 23:00:00,890,215496.601327,46724.747919,3171.432005,Single-Family Detached
175196,2018-12-31 23:15:00,890,215496.601327,40908.034600,2770.220917,Single-Family Detached
175197,2018-12-31 23:30:00,890,215496.601327,38722.802210,3117.436789,Single-Family Detached
175198,2018-12-31 23:45:00,890,215496.601327,37410.936383,3147.945296,Single-Family Detached


In [16]:
result = my_run.savings.savings_shape(upgrade_id=2, enduses=['out.electricity.total.energy_consumption'],
                             group_by=['geometry_building_type_recs'],
                             restrict=[('in.state', ['VT'])],
                             get_quartiles=True,
                             annual_only=True,
                             get_query_only=False)
print(result)

   sample_count    units_count  \
0            89   21549.660133   
1           165   39951.617100   
2           147   35593.258871   
3            54   13075.074687   
4           890  215496.601327   

   electricity.total.energy_consumption__baseline  \
0                                    1.268281e+08   
1                                    2.295646e+08   
2                                    2.067916e+08   
3                                    7.938956e+07   
4                                    1.678065e+09   

   electricity.total.energy_consumption__savings  \
0                                   2.677807e+06   
1                                   2.316868e+07   
2                                   1.868731e+07   
3                                   6.101988e+06   
4                                   1.047801e+08   

  electricity.total.energy_consumption__savings__quartiles  \
0  [-5.861421400000836, 0.0, 31.944746630000054, ...         
1  [-14.067411359999824, 0.0, 21.622132

In [17]:
my_run.save_cache()

INFO:buildstock_query.query_core:No new queries to save.
